In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv


In [2]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

train=pd.read_csv('../input/titanic/train.csv')
test=pd.read_csv('../input/titanic/test.csv')
print(len(train))
print(len(test))

891
418


In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
missing_train = train.isnull().sum()
missing_train = pd.DataFrame({'missing': missing_train})
missing_train = missing_train.query('missing != 0')
missing_train

,missing
Age,177
Cabin,687
Embarked,2


In [6]:
train['Age']= train['Age'].fillna(train['Age'].mean())
train['Cabin']= train['Cabin'].fillna('none')
print(train['Embarked'].value_counts())
train['Embarked']= train['Embarked'].fillna('S')
train.isnull().sum()

S    644
C    168
Q     77
Name: Embarked, dtype: int64


PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [7]:
missing_test = test.isnull().sum()
missing_test = pd.DataFrame({'missing': missing_test})
missing_test = missing_test.query('missing != 0')
missing_test

,missing
Age,86
Fare,1
Cabin,327


In [8]:
test['Age']=test['Age'].fillna(test['Age'].mean())
test['Fare']=test['Fare'].fillna(test['Fare'].mean())
test['Cabin']=test['Cabin'].fillna('none')
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [9]:
#一般化線形モデルを構築。目的変数が2値（生存 or not）なので、二項分布sm.families.Binomial()を選択。
#文字列情報かつ一意の値は説明変数に含めない→Name・Ticket。欠損値の多かった Cabinも含めない
model = smf.glm('Survived ~Pclass+Sex+Age+SibSp+Parch+Fare+Embarked',
                data=train, family=sm.families.Binomial()).fit()

In [10]:
#予測を実行
pred= model.predict(test)
pred.head() #O/Pは生存確率

0    0.105939
1    0.351786
2    0.107384
3    0.098410
4    0.573493
dtype: float64

In [11]:
#提出用に、値を0か1の整数値に変換
pred = np.round(pred.fillna(0.0)).astype(int)
pred.head()

0    0
1    0
2    0
3    0
4    1
dtype: int64

In [12]:
#提出用データ整形(最後の.Tで行列入れ替え)
my_pred=pd.DataFrame([test['PassengerId'], pred], index=['PassengerId','Survived']).T
my_pred.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [13]:
#CSV書き出し
my_pred.to_csv('my_predict_titanic.csv', index=False)